In [ ]:
import numpy as np
from read import df
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
# E.D.A. relevant correlations
corrs = df.corr().loc[:, 'BPXSY1'].dropna()
# print(corrs[corrs > 0.1])

# Analysis variables, only those significantly correlated are kept
# as well as gender, pregnancy status, and race
bp = ['BPXSY1']
body_measurements = ['BMXWAIST']
demos = ['RIDAGEYR', 'RIAGENDR', 'RIDEXPRG', 'RIDRETH1']
analysis_columns = bp + body_measurements + demos

df_analysis = df.loc[:, analysis_columns].dropna()

# Pregnancy ended up not being an interesting variable
#sns.catplot(df, x = 'RIDEXPRG', y = 'BPXSY1')
analysis_columns.remove('RIDEXPRG')

# Final cleaned data frame
df_analysis = df.loc[:, analysis_columns].dropna()

# Race and gender dictionaries will be useful
race_label_dict = {
                'MA':'Mexican American',
                'OH': 'Other Hispanic',
                'NHW': 'Non-Hispanic White',
                'NHB': 'Non-Hispanic Black',
                'Other': 'Other'
                }
gender_label_dict = {
                'F': 'Female',
                'M': 'Male'
}
races = list(race_label_dict.keys())
genders = list(gender_label_dict.keys())

# Change col names
df_analysis.columns = ['BP', 'WAIST', 'AGE', 'GENDER', 'RACE']

In [ ]:
model_formulas = [
    'BP ~ WAIST',
    'BP ~ AGE + WAIST',
    'BP ~ AGE + WAIST + GENDER',
    'BP ~ AGE + WAIST + GENDER + RACE',
    'BP ~ bs(AGE, 5) + WAIST + GENDER + RACE',
    'BP ~ bs(AGE, 5) + bs(WAIST, 5) + GENDER + RACE',
    #  interactions
    '''BP ~ bs(AGE, 5) + WAIST+ GENDER + RACE +
                (bs(AGE, 5) + WAIST)*(GENDER + RACE)
    ''',
    # Two way interactions
    '''BP ~ bs(AGE, 5) + WAIST + GENDER + RACE +
                (bs(AGE, 5) + WAIST)*(GENDER + RACE) +
                (bs(AGE, 5) + WAIST)*GENDER*RACE
    '''
 ]

aics = np.zeros(len(model_formulas))
for i,f in enumerate(model_formulas):
    m_curr = sm.OLS.from_formula(f, df_analysis)
    r_curr = m_curr.fit()
    aics[i] = r_curr.aic

best_model = model_formulas[np.argmin(aics)]

m_best = sm.OLS.from_formula(best_model, df_analysis)
r_best = m_best.fit()

In [ ]:
# Model summary
r_best.summary()

In [ ]:
# The data below will be used to graph
# systolic bp at different combination
# # of factors
test_df = df_analysis.copy()

# Restrict to only a few observations
test_df = test_df.iloc[0:50, :]

# Distribute age from 18 - 50 in even 50 evenly split numbers
test_df['AGE'] = np.linspace(18, 80, 50)
test_df = test_df.drop('BP', axis = 1)

# Replace waist circumference with its average
test_df['WAIST'] = test_df['WAIST'].mean()

def plot_by_race(model_r, gen):
    test_df_curr = test_df.copy()
    test_df_curr['GENDER'] = gen
    for r in races:
        test_df_curr['RACE'] = r
        yh = model_r.predict(test_df_curr)
        _ = sns.lineplot(data = test_df_curr, x = 'AGE', y = yh, label = race_label_dict[r])
        _ = _.set(title = f"Systolic Blood Pressure by Race ({gender_label_dict[gen]})",
                  xlabel = 'Age (years)',
                  ylabel = 'SBP (mm Hg)')
    plt.grid()

def plot_by_gender(model_r, race):
    test_df_curr = test_df.copy()
    test_df_curr['RACE'] = race
    for gender in genders:
        test_df_curr['GENDER'] = gender
        yh = model_r.predict(test_df_curr)
        _ = sns.lineplot(data = test_df_curr, x = 'AGE', y = yh, label = gender_label_dict[gender])
        _ = _.set(title = f"Systolic Blood Pressure by Gender ({race_label_dict[race]})",
                  xlabel = 'Age (years)',
                  ylabel = 'SBP (mm Hg)')
    plt.grid()

def plot_by_waist(model_r, race, gender):
    test_df_curr = test_df.copy()
    test_df_curr['RACE'] = race
    test_df_curr['GENDER'] = gender
    for waist in [80, 120]:
        test_df_curr['WAIST'] = waist
        yh = model_r.predict(test_df_curr)
        _ = sns.lineplot(data = test_df_curr, x = 'AGE', y = yh, label = f'Waist: {waist}')
        _ = _.set(title = f"Systolic Blood Pressure by Waist ({race_label_dict[race]}, {gender_label_dict[gender]})",
                  xlabel = 'Age (years)',
                  ylabel = 'SBP (mm Hg)')
    plt.grid()

def plot_by_race_difference(model_r, race_1, race_2):
    test_df_curr = test_df.copy()
    test_df_curr['GENDER'] = 'M'
    test_df_curr['RACE'] = race_1
    yh_1 = model_r.predict(test_df_curr)
    test_df_curr['RACE'] = race_2
    yh_2 = model_r.predict(test_df_curr)
    _ = sns.lineplot(data = test_df_curr, x = 'AGE', y = yh_1 - yh_2)
    _ = _.set(title = f"Difference in Systolic Blood Pressure ({race_label_dict[race_1]} - {race_label_dict[race_2]})",
                xlabel = 'Age (years)',
                ylabel = 'SBP (mm Hg)')
    plt.grid()


In [ ]:
## Males
plot_by_race(r_best, 'M')
plt.savefig('img/1.png')
plt.clf()

## Females
plot_by_race(r_best, 'F')
plt.savefig('img/2.png')

In [ ]:
# By gender plots

## Mexican Americans
plot_by_gender(r_best, 'MA')
plt.savefig('img/3.png')
plt.clf()

## Non-Hispanic Black
plot_by_gender(r_best, 'NHB')
plt.savefig('img/4.png')
plt.clf()

## Non-Hispanic White
plot_by_gender(r_best, 'NHW')
plt.savefig('img/5.png')
plt.clf()

## Other Hispanic
plot_by_gender(r_best, 'OH')
plt.savefig('img/6.png')
plt.clf()

## Other Race
plot_by_gender(r_best, 'Other')
plt.savefig('img/7.png')

In [ ]:
# Difference in pressure among races

## MA vs NHB
plot_by_race_difference(r_best, 'MA', 'NHB')
plt.savefig('img/8.png')
plt.clf()

## MA vs NHW
plot_by_race_difference(r_best, 'MA', 'NHW')
plt.savefig('img/9.png')
plt.clf()

## MA vs OH
plot_by_race_difference(r_best, 'MA', 'OH')
plt.savefig('img/10.png')
plt.clf()

## MA vs Other
plot_by_race_difference(r_best, 'MA', 'Other')
plt.savefig('img/11.png')
plt.clf()

## OH vs NHW
plot_by_race_difference(r_best, 'OH', 'NHW')
plt.savefig('img/12.png')
plt.clf()

## OH vs NHB
plot_by_race_difference(r_best, 'OH', 'NHB')
plt.savefig('img/13.png')
plt.clf()

## OH vs Other
plot_by_race_difference(r_best, 'OH', 'Other')
plt.savefig('img/14.png')
plt.clf()

## NHW vs NHB
plot_by_race_difference(r_best, 'NHW', 'NHB')
plt.savefig('img/15.png')
plt.clf()

## NHW vs Other
plot_by_race_difference(r_best, 'NHW', 'Other')
plt.savefig('img/16.png')
plt.clf()

## NHB vs Other
plot_by_race_difference(r_best, 'NHB', 'Other')
plt.savefig('img/17.png')